In [13]:
import requests
import pandas as pd
from prophet import Prophet

In [14]:
# Function create a dataframe by fetching the current data.import requests
def fetch_scheme_nav_data(scheme_code):
    api_url = f"https://api.mfapi.in/mf/{scheme_code}"
    response = requests.get(api_url)
    
    if response.status_code == 200:
        json_data = response.json()
        nav_data = json_data.get('data', [])
        
        if nav_data:
            nav_df = pd.DataFrame(nav_data)
            nav_df['date'] = pd.to_datetime(nav_df['date'], format='%d-%m-%Y')
            nav_df['nav'] = pd.to_numeric(nav_df['nav'], errors='coerce')
            nav_df['date'] = nav_df['date'].dt.strftime('%d-%m-%Y')
            nav_df = nav_df[::-1]
            return nav_df
        else:
            print("No NAV data available for the scheme.")
            return None
    else:
        print("Failed to fetch JSON data:", response.status_code)
        return None

In [86]:
scheme_code = 100607
nav_df = fetch_scheme_nav_data(scheme_code)
nav_df

,date,nav
1706,19-06-2006,0.0
1705,20-06-2006,0.0
1704,21-06-2006,0.0
1703,22-06-2006,0.0
1702,23-06-2006,0.0
...,...,...
4,18-06-2013,0.0
3,20-06-2013,0.0
2,09-07-2013,0.0
1,24-07-2013,0.0


In [45]:
# Function to predcit and return the predcited dataframe
def predict_nav(nav_df):
    
    # Check if the dataframe is small
    if len(nav_df) < 100:
        #print("Input dataframe is too small. Returning null dataframe.")
        return pd.DataFrame(columns=['ds', 'yhat'])
    # Step 1: Preprocess the data
    # Ensure 'date' column is in datetime format
    nav_df['ds'] = pd.to_datetime(nav_df['date'], format='%d-%m-%Y')
    # Rename 'nav' column to 'y' as required by Prophet
    nav_df.rename(columns={'nav': 'y'}, inplace=True)

    # Step 2: Train a forecasting model
    model = Prophet()
    model.fit(nav_df)

    # Step 3: Make predictions for the next 5 years
    future = model.make_future_dataframe(periods=365*5, freq='D')
    forecast = model.predict(future)
    
    # Filter out dates beyond the range of the original data
    future_dates = forecast[forecast['ds'] > nav_df['ds'].max()]['ds']

    # Create predicted_values DataFrame with only future dates and predicted NAV values
    predicted_values = forecast[forecast['ds'].isin(future_dates)][['ds', 'yhat']].copy()

    return predicted_values

In [87]:
predicted_df = predict_nav(nav_df)
predicted_df

,ds,yhat
1707,2013-09-27,0.0
1708,2013-09-28,0.0
1709,2013-09-29,0.0
1710,2013-09-30,0.0
1711,2013-10-01,0.0
...,...,...
3527,2018-09-21,0.0
3528,2018-09-22,0.0
3529,2018-09-23,0.0
3530,2018-09-24,0.0


In [88]:
# Function which will return the scores based on the predicted value
def calculate_returns(predicted_nav_df):
    
    
    if predicted_nav_df.empty:  # Check if the dataframe is empty
        return {}  # Return an empty dataframe if it's empty
    # Step 1: Sort the dataframe by date
    
    predicted_nav_df_sorted = predicted_nav_df.sort_values(by='ds')

    # Step 2: Calculate 1-year, 3-year, and 5-year returns
    def calculate_returns(nav_values, dates, years):
        returns = {}
        for year in years:
            # Calculate the NAV at the beginning and end of the period
            end_nav = nav_values.iloc[-1]
            start_date = max(dates) - pd.DateOffset(years=year)
            start_nav = nav_values[dates >= start_date].iloc[0]
                        # If start_nav is zero, replace it with 1
            if start_nav == 0:
                start_nav = 1
            # Calculate the return using the CAGR formula
            cagr = (end_nav / start_nav) ** (1/year) - 1
            returns[f"{year}-Year Return"] = cagr * 100  # Convert to percentage
        return returns

    # Extract NAV values and dates
    nav_values = predicted_nav_df_sorted['yhat']
    dates = predicted_nav_df_sorted['ds']

    # Calculate returns
    years = [1, 3, 5]
    returns = calculate_returns(nav_values, dates, years)

    return returns

In [71]:
returns = calculate_returns(predicted_df)
for year, return_value in returns.items():
    print(f"{year}: {return_value:.2f}%")

1-Year Return: 2.96%
3-Year Return: 2.57%
5-Year Return: 2.51%


In [72]:
print(type(returns))

<class 'dict'>


In [95]:
# Function to create a dataframe with funds and scores
def calculate_scores_for_scheme_codes(scheme_codes, year):
    # Create an empty list to store the results
    results = []

    # Iterate over the list of scheme codes
    for scheme_code in scheme_codes:
        # Fetch NAV data for the scheme
        nav_df = fetch_scheme_nav_data(scheme_code)

        if nav_df is not None:
            # Predict NAV values
            predicted_nav_df = predict_nav(nav_df)

            # Calculate returns
            returns = calculate_returns(predicted_nav_df)
            print(scheme_code)
            if not returns:
                results.append({
                    'Scheme Code': scheme_code,
                    '1-Year Return': -1,
                    '3-Year Return': -1,
                    '5-Year Return': -1
                })
            else:
                # Append scheme code and returns to the results list as a dictionary
                results.append({
                    'Scheme Code': scheme_code,
                    '1-Year Return': returns['1-Year Return'],
                    '3-Year Return': returns['3-Year Return'],
                    '5-Year Return': returns['5-Year Return']
                })
        else:
            print(f"Failed to fetch NAV data for scheme code {scheme_code}. Skipping...")

    # Create DataFrame from the list of dictionaries
    results_df = pd.DataFrame(results)

    returns = {}
    for index, row in results_df.iterrows():
        scheme_code = row['Scheme Code']
        if year == 1:
            returns[scheme_code] = row['1-Year Return']
        elif year == 3:
            returns[scheme_code] = row['3-Year Return']
        elif year == 5:
            returns[scheme_code] = row['5-Year Return']
        else:
            print("Invalid year! Please provide 1, 3, or 5 for year.")
            return None
    return returns


In [96]:
# Example usage:
scheme_codes = [
    101094, 109264, 100477, 100065, 109255, 101551, 101095, 111585, 100684, 110606,
    100999, 100254, 100614, 100315, 100218, 101672, 115399, 100682, 100494, 100363,
    100387, 101160, 101169, 101206, 102201, 100787, 100881, 101003, 100318, 100047,
    101209, 100064, 100355, 100651, 113104, 100609, 100476, 100369, 119334, 113251,
    120491, 115401, 113102, 120488, 102677, 100956, 110608, 103052, 109269, 114630,
    100312, 100650, 118069, 100321, 101185, 100669, 100606, 107288, 100657, 100365,
    100317, 101599, 102055, 101186, 102851, 100325, 100646, 100483, 100220, 101929,
    115402, 100089, 101600, 100316, 100678, 100874, 101365, 100386, 111709, 100853,
    100470, 102261, 100496, 103736, 108479, 115290, 100648, 100873, 101743, 115992,
    101699, 113249, 101864, 100622, 101933, 101224, 100725, 101694, 100372, 100621,
    100504, 101619, 100620, 100052, 100943, 113063, 103051, 100971, 100484, 100360,
    100859, 100608, 100613, 100623, 100748, 117141, 103048, 100644, 100480, 100041,
    102676, 100487, 101691, 100954, 100803, 101481, 101098, 113096, 100049, 101698,
    101314, 119415, 100503, 101229, 102675, 101482, 100781, 142245, 100485, 100285,
    100351, 100875, 100370, 101667, 100968, 100087, 104489, 104635, 101295, 100055,
    112214, 100495, 102065, 100353, 115398, 100475, 100749, 101318, 100122, 100782,
    100740, 100059, 100499, 100899, 101282, 102753, 100751, 101127, 100291, 100610,
    100051, 100472, 102431, 100780, 100786, 101042, 100593, 100868, 101430, 101692,
    100829, 122390, 100838, 125305, 100723, 101228, 101351, 101001, 100724, 111590,
    133839, 101471, 100952, 122389, 101951, 100783, 101605, 100712, 100123, 104726,
    100802, 111589, 100282, 101081, 104488, 100711, 133517, 101696, 122388, 100418,
    115270, 101591, 109942, 101315, 104728, 100717, 100084, 100350, 100645, 100416,
    101000, 101491, 100497, 101635, 131579, 100641, 100474, 100664, 101222, 100046,
    112277, 100120, 101083, 102030, 101593, 124303, 100915, 102202, 100053, 100970,
    112680, 100357, 102849, 101530, 112369, 101687, 102752, 100928, 102262, 100323,
    100063, 112216, 101742, 101350, 104636, 100301, 100656, 100663, 101480, 124233,
    101934, 100917, 119340, 119399, 100082, 101281, 101597, 100639, 100941, 102260,
    100953, 100086, 102066, 100319, 101049, 101364, 122640, 112322, 102257, 100278,
    139534, 113142, 112712, 104723, 100617, 119400, 100081, 112211, 112278, 109938,
    139535, 102061, 100044, 139344, 109945, 101372, 100856, 100929, 101950, 102252,
    100292, 107589, 133838, 101547, 119416, 100927, 100602, 101548, 100668, 100597,
    109940, 100359, 104492, 139538, 104725, 100383, 125711, 100048, 101361, 100262,
    100175, 101065, 112213, 100601, 101805, 100264, 101862, 100630, 101693, 100348,
    100364, 104485, 100604, 100038, 113036, 109950, 113062, 103490, 100600, 109941,
    100354, 100290, 140463, 100366, 100836, 100414, 100998, 115991, 100289, 100832,
    124305, 102172, 100172, 100173, 112679, 100590, 109946, 100481, 100489, 100174,
    101066, 100643, 109939, 100367, 100171, 100176, 100631, 101362, 112682, 101363,
    100088, 100870, 101702, 100362, 107657, 102751, 100611, 100037, 107648, 101304,
    109936, 100876, 103491, 100612, 115995, 116424, 107646, 135601, 100062, 119333,
    100283, 100281, 139539, 100589, 101697, 104490, 100222, 101589, 101636, 100482,
    100871, 135600, 101932, 100784, 100491, 100279, 101113, 101114, 100801, 100119,
    100997, 100286, 100831, 118484, 125306, 100358, 100298, 100299, 101922, 100085,
    101303, 102251, 101231, 100280, 133836, 101935, 100067, 112323, 112683, 133520,
    100596, 100079, 100121, 100361, 101349, 101618, 112648, 133837, 101317, 100080,
    100352, 115994, 104483, 100865, 103050, 100042, 142246, 109935, 100261, 100835,
    139537, 107649, 102433, 101609, 100034, 100058, 111710, 109934, 103780, 100263,
    107647, 120086, 100638, 100077, 101685, 104727, 112645, 135599, 100640, 102058,
    100057, 101316, 143603, 100828, 112408, 102848, 100381, 112212, 100078, 100385,
    116484, 100619, 100750, 100800, 101313, 119354, 100124, 113252, 113253, 104729,
    100265, 100837, 100043, 112210, 101588, 107656, 112647, 101952, 100415, 112681,
    100940, 101306, 101744, 100356, 101144, 101659, 100061, 112684, 101431, 101594,
    101686, 101688, 120084, 100177, 113254, 104484, 104486, 100313, 100851, 100603,
    100607, 100785, 100033, 102259, 122612, 101184, 100858, 101606, 112217, 109937,
    109949, 100869, 100368, 113103, 100833, 101305, 101701, 102258, 101617, 114633,
    116485, 138254, 100493, 105024, 100382, 100054, 101598, 102056, 100056, 112215,
    100284, 115993, 104724, 145963, 100066, 100972, 100473, 100486, 101082, 100955,
    100314, 100379, 102673, 101419, 100877, 100878, 100900, 100371, 125337, 100864,
    102174, 102432, 100739, 100741, 133518, 101665, 100605, 110607, 100872, 100349,
    100300, 107579, 101002, 102053, 112354, 101143, 107578, 135598, 100618, 100788,
    101489, 100471, 101230, 104491, 100332, 101490, 100060, 112355, 100090, 109944,
    104722, 100233, 100830, 139536, 102173, 103735, 133488, 100685, 100490, 113250,
    100834, 101700, 100417, 101587, 100498, 101084, 107693, 100934, 102054, 101221,
    115400, 100742, 101165, 103734
]

scores_df = calculate_scores_for_scheme_codes(scheme_codes,1)
#print(scores_df)
scores_df

21:36:44 - cmdstanpy - INFO - Chain [1] start processing
21:36:44 - cmdstanpy - INFO - Chain [1] done processing


101094


21:36:46 - cmdstanpy - INFO - Chain [1] start processing
21:36:47 - cmdstanpy - INFO - Chain [1] done processing


109264


21:36:49 - cmdstanpy - INFO - Chain [1] start processing
21:36:51 - cmdstanpy - INFO - Chain [1] done processing


...

22:15:42 - cmdstanpy - INFO - Chain [1] start processing
22:15:44 - cmdstanpy - INFO - Chain [1] done processing


101165


22:15:46 - cmdstanpy - INFO - Chain [1] start processing
22:15:48 - cmdstanpy - INFO - Chain [1] done processing


103734


{101094.0: 46.43610498563169,
 109264.0: 0.13876800418524127,
 100477.0: 15.35910802382887,
 100065.0: 6.187502190226968,
 109255.0: -0.08677308053701704,
 101551.0: 9.265841245967765,
 101095.0: 46.60439870347,
 111585.0: 5.483636722458685,
 100684.0: 11.599835388938606,
 110606.0: 15.100007112694946,
 100999.0: 6.719602141324343,
 100254.0: 12.606965365563362,
 100614.0: 6.393776133564066,
 100315.0: 5.602814298426462,
 100218.0: -2.5701905002203818,
 101672.0: 14.922681707470996,
 115399.0: 0.061784951022758605,
 100682.0: 9.192097877347894,
 100494.0: 2.7175098398876774,
 100363.0: 8.02369219900585,
 100387.0: 5.739743944713971,
 101160.0: -1.0,
 101169.0: 8.208599257045602,
 101206.0: 3.3786380001595617,
 102201.0: 2.2348537874699925,
 100787.0: 2.7219110325761964,
 100881.0: 0.7030242425035915,
 101003.0: 5.959090324563787,
 100318.0: 5.6890519367388315,
 100047.0: 3.7176884838771462,
 101209.0: 20.234636934595596,
 100064.0: 7.510324400000834,
 100355.0: 10.45143716912602,
 1006

In [97]:
scores_df

{101094.0: 46.43610498563169,
 109264.0: 0.13876800418524127,
 100477.0: 15.35910802382887,
 100065.0: 6.187502190226968,
 109255.0: -0.08677308053701704,
 101551.0: 9.265841245967765,
 101095.0: 46.60439870347,
 111585.0: 5.483636722458685,
 100684.0: 11.599835388938606,
 110606.0: 15.100007112694946,
 100999.0: 6.719602141324343,
 100254.0: 12.606965365563362,
 100614.0: 6.393776133564066,
 100315.0: 5.602814298426462,
 100218.0: -2.5701905002203818,
 101672.0: 14.922681707470996,
 115399.0: 0.061784951022758605,
 100682.0: 9.192097877347894,
 100494.0: 2.7175098398876774,
 100363.0: 8.02369219900585,
 100387.0: 5.739743944713971,
 101160.0: -1.0,
 101169.0: 8.208599257045602,
 101206.0: 3.3786380001595617,
 102201.0: 2.2348537874699925,
 100787.0: 2.7219110325761964,
 100881.0: 0.7030242425035915,
 101003.0: 5.959090324563787,
 100318.0: 5.6890519367388315,
 100047.0: 3.7176884838771462,
 101209.0: 20.234636934595596,
 100064.0: 7.510324400000834,
 100355.0: 10.45143716912602,
 1006

In [98]:
# Sort the dictionary based on values in ascending order
sorted_scores = dict(sorted(scores_df.items(), key=lambda item: item[1]))

print("Sorted scores:")
for key, value in sorted_scores.items():
    print(f"Scheme Code: {key}, Score: {value}")

Sorted scores:
Scheme Code: 100590.0, Score: -243.51721505865368
Scheme Code: 100802.0, Score: -138.7854024714895
Scheme Code: 100606.0, Score: -100.0
Scheme Code: 112682.0, Score: -100.0
Scheme Code: 112681.0, Score: -100.0
Scheme Code: 112684.0, Score: -100.0
Scheme Code: 100607.0, Score: -100.0
Scheme Code: 100059.0, Score: -72.47678715587334
Scheme Code: 100783.0, Score: -64.93237166534458
Scheme Code: 112217.0, Score: -63.62775598770432
Scheme Code: 112216.0, Score: -63.4864232494564
Scheme Code: 112215.0, Score: -63.350314454115185
Scheme Code: 100749.0, Score: -49.2011394610764
Scheme Code: 100748.0, Score: -47.50837300476923
Scheme Code: 112355.0, Score: -46.55510594403567
Scheme Code: 112369.0, Score: -45.75607400950381
Scheme Code: 117141.0, Score: -42.652540630664916
Scheme Code: 112278.0, Score: -39.20264150484067
Scheme Code: 112322.0, Score: -34.214485609678505
Scheme Code: 112214.0, Score: -19.092827151624792
Scheme Code: 101693.0, Score: -14.303600848211317
Scheme Code: